<a href="https://colab.research.google.com/github/Khaimin6/CTF_solution/blob/main/07_slow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 Slow - OMG, It's slow.

- somehow as a learning note for multiprocessing, threading and ThreadPoolExecutor

In [ ]:
!apt install -y ncat

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblua5.3-0 libpcap0.8
The following NEW packages will be installed:
  liblua5.3-0 libpcap0.8 ncat
0 upgraded, 3 newly installed, 0 to remove and 1 not upgraded.
Need to get 386 kB of archives.
After this operation, 1,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblua5.3-0 amd64 5.3.6-1build1 [140 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 ncat amd64 7.91+dfsg1+really7.80+dfsg1-2ubuntu0.1 [101 kB]
Fetched 386 kB in 1s (339 kB/s)
Selecting previously unselected package libpcap0.8:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libpcap0.8_1.10.1-4ubuntu1.22.04.1_amd64.deb ...
Unpacking 

In [ ]:
!ncat ctf.hackme.quest 7708

Tips: flag has no lowercase or space, and should match this regex: FLAG\{[0-9A-Z_]+\}
What is your flag? FLAG{ABCD}
Bye

^C


In [ ]:
# Source - https://stackoverflow.com/a
# Posted by Jason Orendorff, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-20, License - CC BY-SA 4.0

import socket
import time

def netcat(hostname, port, content, last_done, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    start_time = time.time()

    while 1:
        data = s.recv(128)
        if len(data) == 0:
            break
        # print("Received:", repr(data))

    end_time = time.time()
    connection_time = end_time - start_time

    if connection_time > last_done.value:
      last_done.value = connection_time
      flag.value = c

    s.close()


h:short, i:int, d:double

In [ ]:
import multiprocessing as mp

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content = lambda *data: netcat("ctf.hackme.quest", 7708, *data)

# required to declare shared variable
flag = mp.Array('c', 40)


def try_char():
    last_done = mp.Value("d", 0.0)

    workers = [mp.Process(target=send_content, args=(curr_data + i, last_done, flag)) for i in char_set]

    for p in workers:
        p.daemon = True
        p.start()

    for p in workers:
        p.join()

    global curr_data
    curr_data = flag.value.decode('ascii')
    print(curr_data)


In [ ]:
try_char()

In [ ]:
for i in range(10): try_char()

In [ ]:
flag[:]

b''

In [ ]:
import socket
import time

def netcat_th(hostname, port, content, last_done, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    start_time = time.time()

    while 1:
        data = s.recv(128)
        if len(data) == 0:
            break
        # print("Received:", repr(data))

    end_time = time.time()
    connection_time = end_time - start_time

    if connection_time > last_done:
      print(c)
      last_done = connection_time
      flag = c

    s.close()


In [ ]:
import threading as th

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content_th = lambda *data: netcat_th("ctf.hackme.quest", 7708, *data)
flag = b''


def try_char_th():
    last_done = 0.0

    workers = [th.Thread(target=send_content_th, args=(curr_data + i, last_done, flag)) for i in char_set]

    for p in workers:
        p.start()

    for p in workers:
        p.join()

    global curr_data
    curr_data = flag.decode('ascii')


In [ ]:
try_char_th()

In [ ]:
import socket
import time

def netcat_exc(hostname, port, content, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    # s.setblocking(0)

    start_time = time.time()
    while s.recv(128): pass  # proceed when receive null string (end message)
    end_time = time.time()
    connection_time = end_time - start_time

    s.close()

    return content, connection_time

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content_exc = lambda *data: netcat_exc("ctf.hackme.quest", 7708, *data)
flag = b''

with ThreadPoolExecutor(max_workers=40) as exc:

  for _ in range(20):

    future_map = tuple(exc.submit(send_content_exc, curr_data + i, flag) for i in char_set)

    result = dict(_future.result() for _future in as_completed(future_map))

    curr_data = max(result, key=result.get)
    print(curr_data)
